# 常规赛：PALM病理性近视预测基线方案

**赛题简述**

	PALM病理性近视预测常规赛的重点是研究和发展与病理性近视诊断相关的算法。该常规赛的目标是评估和比较在一个常见的视网膜眼底图像数据集上检测病理性近视的自动算法。具体任务是将提供的图像分为病理性近视眼底彩照和非病理性近视眼底彩照，其中，非病理性近视眼底彩照包括正常眼底和高度近视眼底彩照。
    
![](https://ai-studio-static-online.cdn.bcebos.com/049a11b2a0d6459bbea817529d74be64b69ab7357c8f4990bc56a3b830aa6def)


**数据基本标签**

    非病理：0
    病理  ：1

**训练数据集**

文件名称：Train

Train文件夹里有一个fundus_image文件夹和一个Classification.xlsx文件。fundus_image文件夹中数据均为眼底彩照，分辨率为1444×1444，或2124×2056。命名形如N0001.jpg、H0001.jpg、P0001.jpg和V0001.jpg。Classification.xlsx文件中为各眼底图像是否属于病理性近视，属于为1，不属于为0。

> 本次常规赛提供的病理性近视分类金标准是从临床报告中获取，不仅基于眼底彩照，还结合了OCT、视野检查等结果。

**测试数据集**

文件名称：PALM-Testing400-Images 

文件夹里包含400张眼底彩照，命名形如T0001.jpg。

# 一、基线方案

* 划分训练、验证数据集 —— 划分方法是位于`utils`中 `dataset_splits.py`

* 利用PaddleClas快速选择模型进行比赛 —— 注意数据集路径(`txt`、`img`)

* 预测，得到需要提交的文件 -- 通过修改`infer.py`实现

`MobileNetV3_large_x1_0` —— 得分:0.97457

## 1. 解压PaddleClas套件



In [ ]:
%cd /home/aistudio/
# 解压套件
!unzip -oq data/data90342/PaddleClas-release-2.1.zip -d /home/aistudio/
# 修改文件名
!mv PaddleClas-release-2.1 PaddleClas

# 二、比赛数据情况

* 800张带病理性近视分类标注的眼底彩照  -- `Train`

* 400张带标注数据供平台进行模型测试   -- `Test`

> 解压图像数据到dataset目录下

In [ ]:
%cd /home/aistudio/
!unzip -oq data/data93479/常规赛：PALM病理性近视预测.zip -d PaddleClas/dataset/
# 移除不必要的文件夹
!rm -rf PaddleClas/dataset/__MACOSX

# 三、划分数据

> 划分训练数据与验证数据: 训练集比例为0.75

> 可前往`work/dataset_splits.py`中修改对应`train_percent`参数，重新划分数据

In [ ]:
%cd /home/aistudio/
!python work/dataset_splits.py

# 四、选择比赛模型


* 配置文件参数
```
mode: 'train'
ARCHITECTURE:
    name: "MobileNetV3_large_x1_0"

pretrained_model: ""
model_save_dir: "./output/"
classes_num: 2
total_images: 800
save_interval: 10
ls_epsilon: 0.1
validate: True
valid_interval: 10
epochs: 61
topk: 2
image_shape: [3, 1440, 1440]

# mix数据预处理方法
use_mix: False
ls_epsilon: -1

LEARNING_RATE:
    function: 'Cosine'
    params:
        lr: 0.025
        warmup_epoch: 5

OPTIMIZER:
    function: 'Momentum'
    params:
        momentum: 0.9
    regularizer:
        function: 'L2'
        factor: 0.00001

TRAIN:
    batch_size: 16
    num_workers: 0
    file_list: "./dataset/常规赛：PALM病理性近视预测/train_list.txt"
    data_dir: "./dataset/常规赛：PALM病理性近视预测/"
    shuffle_seed: 0
    transforms:
        - DecodeImage:
            to_rgb: True
            channel_first: False
        - RandCropImage:
            size: 1440
        - RandFlipImage:
            flip_code: 1
        - NormalizeImage:
            scale: 1./255.
            mean: [0.485, 0.456, 0.406]
            std: [0.229, 0.224, 0.225]
            order: ''
        - ToCHWImage:



VALID:
    batch_size: 16
    num_workers: 0
    file_list: "./dataset/常规赛：PALM病理性近视预测/val_list.txt"
    data_dir: "./dataset/常规赛：PALM病理性近视预测/"
    shuffle_seed: 0
    transforms:
        - DecodeImage:
            to_rgb: True
            channel_first: False
        - ResizeImage:
            resize_short: 1444
        - CropImage:
            size: 1440
        - NormalizeImage:
            scale: 1.0/255.0
            mean: [0.485, 0.456, 0.406]
            std: [0.229, 0.224, 0.225]
            order: ''
        - ToCHWImage:





```





> 使用 `MobileNetV3_large_x1_0` 模型进行训练

In [ ]:
%cd /home/aistudio/
# 配置文件文件
!cp -u MobileNetV3_large_x1_0.yaml PaddleClas/configs/MobileNetV3/

In [6]:
# 切换工作目录到PaddleClas下
%cd PaddleClas
# 此时运行程序可以直接运行PaddleClas下的程序和文件
# -c 配置训练文件
!python tools/train.py -c configs/MobileNetV3/MobileNetV3_large_x1_0.yaml

# 五、预测结果生成

> 利用infer程序结合上面训练的模型进行预测输出


> 默认的infer预测保存为：将对应分类的图片进行聚类保存

> 调整后的infer_PALM，直接保存预测结果(保存结果为:label-0：保存值-0.0， label-1：保存值-1.0)，无需其他设置和步骤——生成的`csv`文件放在`首页目录`中，可下载用于提交


先将修改后的设配infer程序移动到指定目录下

> 具体修改内容在`work`文件中可查看`infer_PALM.py`文件

In [7]:
%cd /home/aistudio/
# 调整后的infer文件
!cp -u work/infer_PALM.py PaddleClas/tools/infer/

/home/aistudio


In [12]:
%cd /home/aistudio/
!mkdir PaddleClas/output
!mkdir PaddleClas/output/best_model
# 保存下来的训练权重
!cp -u work/best_model/ppcls.pdopt PaddleClas/output/best_model/
!cp -u work/best_model/ppcls.pdparams PaddleClas/output/best_model/

/home/aistudio


利用配置后的infer进行预测


In [8]:
%cd /home/aistudio/PaddleClas/
!python tools/infer/infer_PALM.py \
    -i dataset/常规赛：PALM病理性近视预测/PALM-Testing400-Images \
    --model MobileNetV3_large_x1_0 \
    # --pretrained_model "./output/MobileNetV3_large_x1_0/best_model/ppcls" \
    --pretrained_model "./output/best_model/ppcls"   #使用训练好的模型进行预测
    --use_gpu True \
    --class_num 2 \
    --resize_short=1444 \
    --resize=1440

/home/aistudio/PaddleClas
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
W0806 06:35:58.920734 15823 device_context.cc:362] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0806 06:35:58.925940 15823 device_context.cc:372] device: 0, cuDNN Version: 7.6.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.p

# 结果投票集成

* 简单投票集成学习，这个可以提升效果，尽量选得分高的进行投票。
* 在统计学和机器学习中，集成学习方法使用多种学习算法来获得比单独使用任何单独的学习算法更好的预测性能。
* 使用不同超参数、不同的特征，不同的结构，运行多次模型可得到不同的预测结果。在这里我使用的是简单投票法，取PM Risk 平均值作为最终的预测结果。 预测出多个result后，进行投票，
* 榜单上的成绩，便是用各个预测结果进行投票生成的，

代码如下：


In [ ]:
%cd /home/aistudio/
import pandas as pd

import numpy as np
df1 = pd.read_csv("0.98578.csv")
df2 = pd.read_csv("0.9854.csv")

df4 = pd.read_csv("0.98573.csv")
df5 = pd.read_csv("0.98534.csv")
dfs=[df1,df2,df4,df5]
PM_Risk=[]
File_Name=[]

for i in range(len(df4)):
    File_Name.append(dfs[0]['FileName'][i])
    avg=(sum(np.array(dfs[x]['PM Risk'][i]) for x in range(len(dfs))))/len(dfs)
  
    
    PM_Risk.append(avg)

submission = pd.DataFrame(data={
                            "FileName": File_Name,
                            "PM Risk": PM_Risk
                        })
submission=submission.sort_values(by='FileName')
submission.to_csv("Classification_Results.csv", index=False)
    


> 参考了红百黑大佬的基线，但是我并没有做出很好的优化

> 项目链接：https://aistudio.baidu.com/aistudio/projectdetail/1968869?channelType=0&channel=0